In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import math
import numpy as np
import os

from torch.utils.data import Dataset

#### Transformer Encoder

In [18]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    

class Transformer(nn.Module):
    def __init__(self, d_model, heads, num_layers,dropout, max_len, output_classes):
        super().__init__()
        self.encoder_layer= nn.TransformerEncoderLayer(d_model=d_model, nhead=heads)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.positional_encoder = PositionalEncoding(d_model=d_model, dropout=dropout, max_len=max_len)
        self.linear = nn.Linear(in_features=d_model, out_features=output_classes)
        
    def forward(self, x):
        x1 = self.positional_encoder(x)
        x2 = self.encoder(x1)
        out = self.linear(x2)
        return out
        


kinematic_features = 22 #dmodel
heads = 11
num_layers = 6
dropout = 0.1
max_len = 100
output_classes = 7

src = torch.rand(10, 32, kinematic_features)

model = Transformer(kinematic_features,heads,num_layers,dropout,max_len,output_classes)

out = model(src)
print(out.shape)

torch.Size([10, 32, 7])


#### Load Data

In [48]:
gesture_labels_directory = './new_dataset/peg_transfer/peg_transfer/labeled/gestures/'
window_size = 10 # 10 frame windows (kinematic data)
enite_dataset = []
counter = 0



class KinematicDataset(Dataset):
    
    def __init__(self,data_directory, window_size):
        
        self.directory = data_directory
        self.window_size = window_size
        (self.X, self.Y) = self._load_data()
        
    def _load_data(self):
        x = []
        y = []
        self.windows_per_file = []
        
        for filename in os.listdir(self.directory):
            f = os.path.join(self.directory, filename)
            # checking if it is a file
            if os.path.isfile(f):
                kinematics_data = pd.read_csv(f)
                kin_data = kinematics_data.loc[:,:-1]
                kin_label = kinematics_data.loc[:,-1]
                
                x.append(kin_data.values)
                y.append(kin_label.values)
                
                if len(self.windows_per_file) == 0:
                    self.windows_per_file = [len(kin_label) - self.window_size + 1]
                else:
                    self.windows_per_file.append(self.windows_per_file[-1] + len(kin_label) - self.window_size + 1)
        
        return (x, y)
    
    def __len__(self):
        # this should return the size of the dataset
        return sum(self.windows_per_file)

    def __getitem__(self, idx):
        # this should return one sample from the dataset
        file_index = 0
        while self.windows_per_file[file_index] < idx:
            file_index += 1
        window_start_index = self.windows_per_file[file_index]
        features = self.X[file_index][window_start_index : window_start_index + window_size]
        target = self.y[file_index][window_start_index : window_start_index + window_size]
        return features, target
                    
                    







for filename in os.listdir(gesture_labels_directory):
        counter += 1
        f = os.path.join(gesture_labels_directory, filename)
        # checking if it is a file
        if os.path.isfile(f):
            # print(f)
            kinematics_data = pd.read_csv(f)
            # print(kinematics_data)

print(enite_dataset)


In [ ]:
train_data, test_data = 